In [1]:
#Predictions sur les images test avec le model_image_all.06-1.20.h5

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import metrics

from time import time

from keras.preprocessing.image import ImageDataGenerator
import keras

from keras.models import load_model
from keras.applications.vgg16 import preprocess_input

import requests
import io

In [9]:
#télécharger le model
model = load_model('/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/model_image_all.06-1.20.h5') 

In [10]:
url = "https://raw.githubusercontent.com/JulienJ-44/rakuteam/main/Datasets/X_test_update.csv" 
download = requests.get(url).content

X_test= pd.read_csv(io.StringIO(download.decode('utf-8')))

X_test.head()

,Unnamed: 0,designation,description,productid,imageid
0,84916,Folkmanis Puppets - 2732 - Marionnette Et Théâ...,NaN,516376098,1019294171
1,84917,Porte Flamme Gaxix - Flamebringer Gaxix - 136/...,NaN,133389013,1274228667
2,84918,Pompe de filtration Speck Badu 95,NaN,4128438366,1295960357
3,84919,Robot de piscine électrique,<p>Ce robot de piscine d&#39;un design innovan...,3929899732,1265224052
4,84920,Hsm Destructeur Securio C16 Coupe Crois¿E: 4 X...,NaN,152993898,940543690


In [11]:
X_test = X_test.drop(["Unnamed: 0", "designation", "description"], axis=1)
X_test["image_name"] = "image_" + X_test.imageid.map(str)+ "_product_" + X_test.productid.map(str) + ".jpg"
X_test["image_name"] = X_test["image_name"].astype(str)

#changer le path au path de vos images
X_test['fullpath']= '/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/images/image_test/'+ X_test['image_name']
X_test.head()

,productid,imageid,image_name,fullpath
0,516376098,1019294171,image_1019294171_product_516376098.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
1,133389013,1274228667,image_1274228667_product_133389013.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
2,4128438366,1295960357,image_1295960357_product_4128438366.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
3,3929899732,1265224052,image_1265224052_product_3929899732.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
4,152993898,940543690,image_940543690_product_152993898.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...


In [14]:
import tensorflow as tf
from tqdm import tqdm

X_test_path = X_test.fullpath

X_test_images = []
for filepath in tqdm(X_test_path):
    # Read the file
    im = tf.io.read_file(filepath)
    # Decode the file
    im = tf.image.decode_jpeg(im, channels=3)
    # Resizing
    im = tf.image.resize(im, size=(224,224))
    X_test_images.append([im])
    
X_test_images = tf.concat(X_test_images, axis=0)

100%|██████████| 13812/13812 [02:34<00:00, 89.44it/s] 


In [15]:
y_pred_proba = model.predict(X_test_images, batch_size=64)
y_pred_proba

array([[8.9791559e-02, 4.3010859e-05, 4.0371638e-02, ..., 6.2395293e-02,
        6.7115322e-02, 3.3172712e-02],
       [7.6006686e-20, 1.8089962e-15, 1.0263491e-21, ..., 5.2812497e-25,
        2.1603970e-24, 7.3932701e-05],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 3.2565012e-21,
        2.1627671e-31, 9.0709923e-38],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 3.0891796e-25,
        6.6281726e-33, 6.5420706e-38],
       [4.6265577e-03, 8.5841055e-04, 3.2817457e-02, ..., 5.9962913e-02,
        3.7757573e-01, 9.4399445e-02],
       [1.9183846e-13, 3.9142112e-18, 2.2399432e-08, ..., 4.0051886e-03,
        6.7978487e-03, 2.3451925e-09]], dtype=float32)

In [18]:
# Label prediction
y_pred_class = tf.argmax(y_pred_proba, axis=-1).numpy()
y_pred_class

array([ 5,  9,  5, ...,  5, 25, 15])

In [19]:
df_tosave=pd.DataFrame(y_pred_proba)
path = '/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/'
df_tosave.to_csv(f'{path}/ypred_proba_model_image_all.06-1.20.csv')